# SMAP to Raster Tutorial with Python
## About
    Currently, Earth Lab is receiving data called 'SMAP data' which is provided in .h5 format. This file format is considered to be "hierarchical" consisting of a multitude of groups and datasets (further explained in Step 2) which are extremely useful in storing and organizing large amounts of data. While this organization is great for the large amount of data being collected, we don't necessarily want to deal with all of the data and may only want to utilize a single dataset within the file. Therefore, it'd be useful to have something that can grab the specific data we want from the file, transform it into a more useable format, and create something (a Raster) which can help display the data we specified.
    
## Objectives
    1) Read in SMAP data file (in .h5 format)
    2) Extract specified data
    3) Create a raster object with respect to this data

In [1]:
#Install the h5py, gdal, and numpy packages
#Note: Ensure you already have python and pip installed on your system
!pip install h5py
!pip install gdal==1.11.2
!pip install numpy -U

Requirement already up-to-date: numpy in c:\users\matto_000\anaconda2\lib\site-packages


# Step 1: Getting our data
NOTE: While the Python script is a relatively small file and will download quickly, the h5 file is NOT small and will take a few minutes to fully download. You'll know this download is completed when the asterisk in the 'In' parameter has become a numeric value and there is output below the cell.

In [2]:
import urllib
import ftplib
import sys

#Useful information about the system being used
print(sys.version)
print '----------------------------------------------------------------------------------------------'

#Download of python helper script which will be used later
pythonfileDL = urllib.URLopener()
pythonfileDL.retrieve('https://svn.osgeo.org/gdal/trunk/gdal/swig/python/scripts/gdal_merge.py',
                  'gdal_merge.py')

#Download of SMAP data in .h5 format
ftp = ftplib.FTP("n5eil01u.ecs.nsidc.org")
ftp.login()

path = "SAN/SMAP/SPL4SMGP.001/2015.03.31/"
ftp.cwd(path)

filename = "SMAP_L4_SM_gph_20150331T013000_Vb1010_001.h5"
ftp.retrbinary("RETR " + filename, open(filename, "wb"). write)

ftp.quit()

2.7.11 |Anaconda 2.4.1 (64-bit)| (default, Feb 16 2016, 09:58:36) [MSC v.1500 64 bit (AMD64)]
----------------------------------------------------------------------------------------------


'221-You have transferred 139218579 bytes in 1 files.\n221-Total traffic for this session was 139219589 bytes in 1 transfers.\n221-Thank you for using the FTP service on ftphost.\n221 Goodbye.'

# Step 2: Viewing our data
It's going to be necessary for us to see what the data we're trying to work with consists of

SMAP files (in .h5 format) consist of 'Groups' and 'Datasets'

Datasets are multidimensional arrays of a homogenous type

Groups are a container structure which hold numerous datasets

In [3]:
#This block of code will print our all of the datasets within a specific group within our SMAP file
#Let's say we have a group called 'Geophysical_Data' that we wish to inspect

import h5py
import numpy as np

h5file = h5py.File('SMAP_L4_SM_gph_20150331T013000_Vb1010_001.h5', 'r') #Replace <Filename> with the name of your h5 file
group = h5file.get('Geophysical_Data') #Replace Geophysical_Data with the name of the group you want to inspect
datasets = np.array(group)
print datasets

[u'surface_pressure' u'heat_flux_ground' u'land_fraction_wilting'
 u'soil_temp_layer5' u'sm_rootzone' u'radiation_longwave_absorbed_flux'
 u'specific_humidity_lowatmmodlay' u'surface_temp' u'temp_lowatmmodlay'
 u'net_downward_longwave_flux' u'snow_mass'
 u'precipitation_total_surface_flux' u'sm_surface_wetness'
 u'sm_profile_wetness' u'snow_depth' u'height_lowatmmodlay'
 u'baseflow_flux' u'sm_profile_pctl' u'land_fraction_saturated'
 u'snowfall_surface_flux' u'land_fraction_unsaturated'
 u'sm_rootzone_wetness' u'leaf_area_index'
 u'radiation_shortwave_downward_flux' u'sm_surface' u'soil_temp_layer1'
 u'heat_flux_sensible' u'soil_temp_layer4' u'soil_temp_layer6'
 u'land_fraction_snow_covered' u'land_evapotranspiration_flux'
 u'vegetation_greenness_fraction' u'sm_profile' u'soil_temp_layer3'
 u'overland_runoff_flux' u'snow_melt_flux' u'heat_flux_latent'
 u'soil_water_infiltration_flux' u'windspeed_lowatmmodlay'
 u'sm_rootzone_pctl' u'net_downward_shortwave_flux' u'soil_temp_layer2']


# Step 3: Using our data to create a Raster
Now that we've decided which group(s) and dataset(s) we wish to use we need to create a Raster object from the data

In [4]:
import h5py
from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr
import numpy as np

#Helper function which handles the conversion from SMAP data to a Raster object
#Function input:  inputFile: SMAP data file
#                 group: The groupt containing the dataset we want to pull data from
#                 dataset: Which specific dataset we want to pull data from
#Function output: A raster image in .tif format consisting of the data we specified in the input
def smap2raster(inputFile, group, dataset):
    
    #Read in the SMAP file in h5 format
    h5File = h5py.File(inputFile, 'r')
    
    #Get the data from the specific group/dataset
    data = h5File.get(group + '/' + dataset)
    lat = h5File.get('cell_lat')
    lon = h5File.get('cell_lon')
    
    #Convert this data into numpy arrays
    np_data = np.array(data)
    np_lat = np.array(lat)
    np_lon = np.array(lon)
    
    #Get the spatial extents of the data
    num_cols = float(np_data.shape[1])
    num_rows = float(np_data.shape[0])
    xmin = np_lon.min()
    xmax = np_lon.max()
    ymin = np_lat.min()
    ymax = np_lat.max()
    xres = (xmax - xmin)/num_cols
    yres = (ymax - ymin)/num_rows
    
    #Set up the transformation necessary to create the raster
    geotransform = (xmin, xres, 0, ymax, 0, -yres)
    
    #Create the raster object with the proper coordinate encoding and geographic transformation
    driver = gdal.GetDriverByName('GTiff')
    raster = driver.Create(dataset+'Raster.tif', int(num_cols), int(num_rows), 1, gdal.GDT_Float32)
    raster.SetGeoTransform(geotransform)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)
    
    #Export and write the data array to the raster
    raster.SetProjection( srs.ExportToWkt() )
    raster.GetRasterBand(1).WriteArray(np_data)
    h5File.close()

#Create an array of the datasets we want to use
#Replace 'snow_mass' and 'snow_depth' with the datasets you want to use
datasets = ['sm_surface_wetness', 'soil_temp_layer2']

#Loop through the datasets and create individual rasters from them
for i in range(0, len(datasets)):
    smap2raster('SMAP_L4_SM_gph_20150331T013000_Vb1010_001.h5', 'Geophysical_Data', datasets[i])

# Step 4: Creating a Raster Stack
After running the above code, you've now created individual raster images (.tif files); the number of which depends on the amount of datasets you used

Now that we have these individual rasters, we want to merge or "stack" them on top of each other

Fortunately, gdal has a python script which accomplishes just this

The result of running this will be a new .tif file called out.tif which is the merging or 'stacking' of the 2 rasters we created in Step 2

In [5]:
%run gdal_merge.py sm_surface_wetnessRaster.tif soil_temp_layer2Raster.tif